# Example: Read 1 Degree Atmosphere (FV3) Replay Data, Store to Zarr

Example using [FV3Dataset](generated/ufs2arco.FV3Dataset.rst) to read data from a single cycle of the replay dataset.

In [1]:
import xarray as xr
from datetime import datetime

In [2]:
import sys
sys.path.append("..")
from ufs2arco import FV3Dataset

## Setup path to read from

In this case, read replay data from the s3 bucket.
Right now, [FV3Dataset](generated/ufs2arco.FV3Dataset.rst) requires a `path_in` callable with 3 entries, so that it can build file paths for a given date (denoting a DA cycle), forecast hours to grab, and file prefixes.

In [3]:
def replay_path(cycles, forecast_hours, file_prefixes):
    
    upper = "s3://noaa-ufs-gefsv13replay-pds/1deg"
    cycles = [cycles] if not isinstance(cycles, list) else cycles

    files = []
    for cycle in cycles:
        this_dir = f"{cycle.year:04d}/{cycle.month:02d}/{cycle.year:04d}{cycle.month:02d}{cycle.day:02d}{cycle.hour:02d}"
        for fp in file_prefixes:
            for fhr in forecast_hours:
                this_file = f"{this_dir}/{fp}{cycle.year:04d}{cycle.month:02d}{cycle.day:02d}{cycle.hour:02d}_fhr{fhr:02d}_control"
                files.append(this_file)
    return [f"{upper}/{this_file}" for this_file in files]

For instance to grab the `bfg_` and `sfg_` files at forecast hour `fhr00` and `fhr06` 
from the DA cycle at 00:00 Jan 1, 1994, this would be used as follows:

In [4]:
cycle = datetime(1994,1,1,0)
replay_path(cycle, [0, 6], ["sfg_", "bfg_"])

['s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/sfg_1994010100_fhr00_control',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/sfg_1994010100_fhr06_control',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/bfg_1994010100_fhr00_control',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/bfg_1994010100_fhr06_control']

However, reading from s3 is super slow with `xarray.open_dataset`.
Luckily we can tell fsspec to cache the files locally before opening, this is denoted by
prepending the names as follows.

See discussion [here](https://discourse.pangeo.io/t/reading-goes-r-s3-netcdfs-from-an-aws-ec2-instance-is-it-possible-to-get-faster-speeds-than-from-my-local-machine/2440/13)
for more info.

In [5]:
def cached_path(cycles, forecast_hours, file_prefixes):
    return [f"simplecache::{u}" for u in replay_path(cycles, forecast_hours, file_prefixes)]

In [6]:
cycle = datetime(1994,1,1,0)
cached_path(cycle, [0, 6], ["sfg_", "bfg_"])

['simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/sfg_1994010100_fhr00_control',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/sfg_1994010100_fhr06_control',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/bfg_1994010100_fhr00_control',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/bfg_1994010100_fhr06_control']

## Use FV3Dataset

Some of that is wrapped up under the hood in [FV3Dataset](generated/ufs2arco.FV3Dataset.rst), we just need to give two inputs:

1. The filename mapping as defined above

2. A configuration yaml file

In [7]:
reader = FV3Dataset(path_in=cached_path, config_filename="config-replay.yaml")

FV3Dataset.__init__: Could not find coords_path_out in config-replay.yaml, using default.


where the config file looks something like...

In [8]:
!cat config-replay.yaml

FV3Dataset:
    file_prefixes:
        - bfg_
        - sfg_

    path_out        : "example-replay-1deg"
    forecast_hours  : [0, 6]

    chunks_in:
        time        : 1
        pfull       : 1
        grid_yt     : -1
        grid_xt     : -1

    chunks_out:
        time        : 1
        pfull       : 127
        grid_yt     : 60
        grid_xt     : 60

    coords:
        - phalf
        - pfull
        - grid_xt
        - grid_yt
        - ak
        - bk

    data_vars:
        - tmp
        - ugrd
        - vgrd
        - dpres
        - delz
        - spfh
        - o3mr
        - soilt1
        - soilt2
        - soilt3
        - soilt4
        - soill1
        - soill2
        - soill3
        - soill4
        - soilw1
        - soilw2
        - soilw3
        - soilw4
        - snod
        - prateb_ave
        - pressfc
        - weasd
        - f10m
        - sfcr
        - land

MOM6Dataset:
    file_prefixes:
        - ocn_
    path_out        : "example-replay-1

We'll use this reader to open a dataset with [.open_dataset()](generated/ufs2arco.FV3Dataset.open_dataset.rst), note that the `fsspec_kwargs` is not necessary for local file reads, but necessary for reading from buckets like s3

In [9]:
%%time
ds = reader.open_dataset(cycle, fsspec_kwargs={"s3":{"anon":True}}, engine="h5netcdf")
ds

/Users/tsmith/miniconda3/envs/ufs2arco/lib/python3.11/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "pfull" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/tsmith/miniconda3/envs/ufs2arco/lib/python3.11/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "pfull" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/tsmith/work/ufs2arco/docs/../ufs2arco/ufsdataset.py:292: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray 

CPU times: user 9.52 s, sys: 4.57 s, total: 14.1 s
Wall time: 1min 1s


<xarray.Dataset>
Dimensions:        (pfull: 127, phalf: 128, time: 2, grid_xt: 384, grid_yt: 192)
Coordinates:
  * pfull          (pfull) float32 0.01278 0.02033 0.03177 ... 993.5 996.2 998.8
  * phalf          (phalf) float32 0.00999 0.01605 0.02532 ... 994.9 997.5 1e+03
    cftime         (time) object 1993-12-31 18:00:00 1994-01-01 00:00:00
  * grid_xt        (grid_xt) float64 0.0 0.9375 1.875 ... 357.2 358.1 359.1
  * grid_yt        (grid_yt) float64 89.28 88.36 87.42 ... -87.42 -88.36 -89.28
  * time           (time) datetime64[ns] 1993-12-31T18:00:00 1994-01-01
    ftime          (time) timedelta64[ns] -1 days +18:00:00 00:00:00
Data variables: (12/182)
    time_iso       (time) object dask.array<chunksize=(1,), meta=np.ndarray>
    lon            (time, grid_yt, grid_xt) float64 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    lat            (time, grid_yt, grid_xt) float64 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    acond          (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    albdo_ave      (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    alnsf          (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
    ...             ...
    rwmr           (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
    snmr           (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
    spfh           (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
    tmp            (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
    ugrd           (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
    vgrd           (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1, 192, 384), meta=np.ndarray>
Attributes:
    dtp:          720.0
    fhzero:       3
    grid:         gaussian
    grid_id:      1
    im:           384
    imp_physics:  8
    jm:           192
    landsfcmdl:   2
    ncld:         8
    nsoil:        4
    source:       FV3GFS

xarray gives a nice view of the data, we can look at a single variable to see how the data are laid out

In [10]:
ds["tmp"]

<xarray.DataArray 'tmp' (time: 2, pfull: 127, grid_yt: 192, grid_xt: 384)>
dask.array<concatenate, shape=(2, 127, 192, 384), dtype=float32, chunksize=(1, 1, 192, 384), chunktype=numpy.ndarray>
Coordinates:
  * pfull    (pfull) float32 0.01278 0.02033 0.03177 ... 993.5 996.2 998.8
    cftime   (time) object 1993-12-31 18:00:00 1994-01-01 00:00:00
  * grid_xt  (grid_xt) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * grid_yt  (grid_yt) float64 89.28 88.36 87.42 86.49 ... -87.42 -88.36 -89.28
  * time     (time) datetime64[ns] 1993-12-31T18:00:00 1994-01-01
    ftime    (time) timedelta64[ns] -1 days +18:00:00 00:00:00
Attributes:
    cell_methods:               time: point
    long_name:                  temperature
    output_file:                dyn
    units:                      K
    max_abs_compression_error:  0.0046844482
    nbits:                      14

## Storing the dataset

The dataset has more variables than we wanted, but these are trimmed when we call [.store_dataset()](generated/ufs2arco.FV3Dataset.store_dataset.rst).

Any arguments beyond the dataset are passed to `xarray.to_zarr`.

By default, dask (which we're accessing through xarray under the hood) stores data to zarr using the chunksizes that are being used currently by dask (i.e., how the data are viewed in memory, currently).
However, we will change this layout based on the `chunks_out` option in our configuration yaml.

In [11]:
reader.chunks_out

{'time': 1, 'pfull': 127, 'grid_yt': 60, 'grid_xt': 60}

In [12]:
reader.store_dataset(ds, mode="w")

Stored dataset at example-replay-1deg/fv3.zarr


Now we can open the zarr store and make sure everything is done right.

In [13]:
xds = xr.open_zarr(reader.data_path)

In [14]:
xds

<xarray.Dataset>
Dimensions:     (time: 2, pfull: 127, grid_yt: 192, grid_xt: 384)
Coordinates:
    cftime      (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime       (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * grid_xt     (grid_xt) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * grid_yt     (grid_yt) float64 89.28 88.36 87.42 ... -87.42 -88.36 -89.28
  * pfull       (pfull) float32 0.01278 0.02033 0.03177 ... 993.5 996.2 998.8
  * time        (time) datetime64[ns] 1993-12-31T18:00:00 1994-01-01
Data variables: (12/26)
    delz        (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    dpres       (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    f10m        (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    land        (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    o3mr        (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    prateb_ave  (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    ...          ...
    soilw4      (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    spfh        (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    tmp         (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    ugrd        (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    vgrd        (time, pfull, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 127, 60, 60), meta=np.ndarray>
    weasd       (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
Attributes:
    dtp:          720.0
    fhzero:       3
    grid:         gaussian
    grid_id:      1
    im:           384
    imp_physics:  8
    jm:           192
    landsfcmdl:   2
    ncld:         8
    nsoil:        4
    source:       FV3GFS

We can see that this only has the data variables we asked for in the config yaml file.

In [15]:
xds.tmp

<xarray.DataArray 'tmp' (time: 2, pfull: 127, grid_yt: 192, grid_xt: 384)>
dask.array<open_dataset-tmp, shape=(2, 127, 192, 384), dtype=float32, chunksize=(1, 127, 60, 60), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * grid_xt  (grid_xt) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * grid_yt  (grid_yt) float64 89.28 88.36 87.42 86.49 ... -87.42 -88.36 -89.28
  * pfull    (pfull) float32 0.01278 0.02033 0.03177 ... 993.5 996.2 998.8
  * time     (time) datetime64[ns] 1993-12-31T18:00:00 1994-01-01
Attributes:
    cell_methods:               time: point
    long_name:                  temperature
    max_abs_compression_error:  0.0046844482421875
    nbits:                      14
    output_file:                dyn
    units:                      K

And we can see this has a different chunking scheme than the original dataset, it has what we asked for in the config yaml.

In [16]:
reader.chunks_out

{'time': 1, 'pfull': 127, 'grid_yt': 60, 'grid_xt': 60}

<div class="alert alert-info">

Note:

This chunking scheme is not necessarily desirable, as the chunk sizes are quite small. This is simply used as an example to illustrate that we have control
over the chunk sizes.

</div>

## Cleanup

In [17]:
from shutil import rmtree

In [18]:
rmtree(reader.data_path)